# MultiplEYE preprocessing

In [5]:
from pathlib import Path
import polars as pl
import re
from sanity import Sanity
from report import check_gaze, check_metadata, check_events, report_to_file as report_meta
from functools import partial
from plot import plot_gaze, plot_main_sequence
from stimulus import load_stimuli, Stimulus


import pymovements as pm

In [ ]:
### Turkisch Data

path_asc_file = "data\\001_TR_TR_1_ET1\\001trtr1.asc"
stimulus_file_path = "C:\\Users\saphi\PycharmProjects\multipleye-preprocessing\data\stimuli_MultiplEYE_TR_TR_Istanbul_1_2025"
logfile_path = f"C:\\Users\saphi\PycharmProjects\multipleye-preprocessing\data\\001_TR_TR_1_ET1\logfiles\EXPERIMENT_LOGFILE_1_001_2025-01-16_1737022316.txt"




In [87]:
### Cuis test data


path_asc_file= "data\\010_ZH_CH_1_ET1\\010zhch1.asc"
stimulus_file_path = "C:\\Users\saphi\PycharmProjects\multipleye-preprocessing\data\stimuli_MultiplEYE_ZH_CH_Zurich_1_2025"
logfile_path = f"C:\\Users\saphi\PycharmProjects\multipleye-preprocessing\data\\010_ZH_CH_1_ET1\logfiles\EXPERIMENT_LOGFILE_1_010_2024-11-15_1731682963.txt"


In [88]:
### Dutch Data
path_asc_file = "data\\017_NL_NL_1_ET1_testrun_1733144369\\017nlnl1.asc"
stimulus_file_path = "C:\\Users\saphi\PycharmProjects\multipleye-preprocessing\data\stimuli_MultiplEYE_NL_NL_Nijmegen_1_2024"
logfile_path = f"C:\\Users\saphi\PycharmProjects\multipleye-preprocessing\data\\017_NL_NL_1_ET1_testrun_1733144369\logfiles\EXPERIMENT_LOGFILE_1_017_2024-12-02_1733144369.txt"

In [ ]:
### Estonian Data


path_asc_file_003 = "data\\003_ET_EE_1_ET1\\003etee1.asc"
logfile_path_003 = f"C:\\Users\saphi\PycharmProjects\multipleye-preprocessing\data\\003_ET_EE_1_ET1\logfiles\EXPERIMENT_LOGFILE_1_003_2025-01-17_1737097063.txt"

path_asc_file_002 = "data\\002_ET_EE_1_ET1\\002etee1.asc"
logfile_path_002 = f"C:\\Users\saphi\PycharmProjects\multipleye-preprocessing\data\\002_ET_EE_1_ET1\logfiles\EXPERIMENT_LOGFILE_1_002_2025-01-16_1737035642.txt"


path_asc_file = "data\\001_ET_EE_1_ET1\\001etee1.asc"
logfile_path = f"C:\\Users\saphi\PycharmProjects\multipleye-preprocessing\data\\001_ET_EE_1_ET1\logfiles\EXPERIMENT_LOGFILE_1_001_2025-01-16_1737028666.txt"


stimulus_file_path = "C:\\Users\saphi\PycharmProjects\multipleye-preprocessing\data\stimuli_MultiplEYE_ET_EE_Tartu_1_2025"



In [9]:
path_asc_file = "data\\003_HR_HR_1_ET1\\003hrhr1.asc"
logfile_path = f"C:\\Users\saphi\PycharmProjects\multipleye-preprocessing\data\\003_HR_HR_1_ET1\logfiles\EXPERIMENT_LOGFILE_1_003_2024-11-07_1730977589.txt"

stimulus_file_path = "C:\\Users\saphi\PycharmProjects\multipleye-preprocessing\data\stimuli_MultiplEYE_HR_HR_Zagreb_1_2025"




In [10]:
### russish Data

path_asc_file = "data\\001_RU_RU_1_ET1\\001ruru1.asc"
stimulus_file_path = "C:\\Users\saphi\PycharmProjects\multipleye-preprocessing\data\stimuli_MultiplEYE_RU_RU_NewYork_1_2025"
logfile_path = f"C:\\Users\saphi\PycharmProjects\multipleye-preprocessing\data\\001_RU_RU_1_ET1\logfiles\EXPERIMENT_LOGFILE_1_001_2025-01-24_1737717269.txt"



Match was found at 5-33: 001_ET_EE_1_ET1\001etee1.asc
Group 1 found at 5-8: 001
Group 2 found at 9-11: ET
Group 3 found at 12-14: EE
Group 4 found at 15-16: 1
Group 5 found at 17-20: ET1
Group 6 found at -1--1: None
Group 7 found at 21-29: 001etee1


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\saphi\\PycharmProjects\\multipleye-preprocessing'

In [15]:
local_file_part = "C:\\Users\saphi\PycharmProjects\multipleye-preprocessing"

sanity = Sanity.load(path_asc_file, local_file_part, stimulus_file_path, logfile_path)
#sanity002 = Sanity.load(path_asc_file_002, local_file_part, stimulus_file_path, logfile_path_002)
#sanity003 = Sanity.load(path_asc_file_003, local_file_part, stimulus_file_path, logfile_path_003)

WindowsPath('C:/Users/saphi/PycharmProjects/multipleye-preprocessing/data/001_TR_TR_1_ET1/001trtr1.asc')

In [ ]:

sanity.get_frame()
sanity.gaze._metadata

In [16]:
sanity.get_frame()
sanity.gaze._metadata


Experiment(sampling_rate=1000.00, screen=Screen(width_px=1341, height_px=1009, width_cm=37, height_cm=28, distance_cm=60, origin=upper left), eyetracker=EyeTracker(sampling_rate=1000.0, left=True, right=False, model='EyeLink 1000 Plus', version='5.09', vendor='EyeLink', mount='Binocular Tower Mount'))


{'weekday': 'Thu',
 'month': 'Jan',
 'day': 16,
 'time': '14:26:20',
 'year': 2025,
 'version_1': 'EYELINK II 1',
 'version_2': 'EYELINK II CL v5.09 Nov 17 2015',
 'resolution': (1341, 1009),
 'tracking_mode': 'CR',
 'sampling_rate': 1000.0,
 'file_sample_filter': '2',
 'link_sample_filter': '1',
 'tracked_eye': 'L',
 'mount_configuration': {'mount_type': 'Binocular Tower Mount',
  'head_stabilization': 'stabilized',
  'eyes_recorded': 'binocular / monocular',
  'short_name': 'BTOWER'},
 'pupil_data_type': 'AREA',
 'version_number': '5.09',
 'model': 'EyeLink 1000 Plus',
 'datetime': datetime.datetime(2025, 1, 16, 14, 26, 20),
 'calibrations': [{'timestamp': '1091276.0',
   'num_points': '9',
   'type': 'P-CR',
   'tracked_eye': 'LEFT'},
  {'timestamp': '1440454.0',
   'num_points': '9',
   'type': 'P-CR',
   'tracked_eye': 'LEFT'},
  {'timestamp': '1973449.0',
   'num_points': '9',
   'type': 'P-CR',
   'tracked_eye': 'LEFT'},
  {'timestamp': '2381613.0',
   'num_points': '9',
   'typ

In [7]:
sanity002.get_frame()
sanity002.gaze._metadata


{'weekday': 'Thu',
 'month': 'Jan',
 'day': 16,
 'time': '15:34:29',
 'year': 2025,
 'version_1': 'EYELINK II 1',
 'version_2': 'EYELINK II CL v5.12 May 12 2017',
 'resolution': (1585, 1199),
 'tracking_mode': 'CR',
 'sampling_rate': 1000.0,
 'file_sample_filter': '2',
 'link_sample_filter': '1',
 'tracked_eye': 'R',
 'mount_configuration': {'mount_type': 'Desktop',
  'head_stabilization': 'stabilized',
  'eyes_recorded': 'monocular',
  'short_name': 'MTABLER'},
 'pupil_data_type': 'AREA',
 'version_number': '5.12',
 'model': 'EyeLink 1000 Plus',
 'datetime': datetime.datetime(2025, 1, 16, 15, 34, 29),
 'calibrations': [{'timestamp': '9129603.0',
   'num_points': '9',
   'type': 'P-CR',
   'tracked_eye': 'RIGHT'},
  {'timestamp': '9201781.0',
   'num_points': '9',
   'type': 'P-CR',
   'tracked_eye': 'RIGHT'},
  {'timestamp': '9448289.0',
   'num_points': '9',
   'type': 'P-CR',
   'tracked_eye': 'RIGHT'},
  {'timestamp': '9481210.0',
   'num_points': '9',
   'type': 'P-CR',
   'tracke

In [8]:
sanity003.get_frame()
sanity003.gaze._metadata

{'weekday': 'Fri',
 'month': 'Jan',
 'day': 17,
 'time': '08:38:11',
 'year': 2025,
 'version_1': 'EYELINK II 1',
 'version_2': 'EYELINK II CL v5.12 May 12 2017',
 'resolution': (1585, 1199),
 'tracking_mode': 'CR',
 'sampling_rate': 1000.0,
 'file_sample_filter': '2',
 'link_sample_filter': '1',
 'tracked_eye': 'R',
 'mount_configuration': {'mount_type': 'Desktop',
  'head_stabilization': 'stabilized',
  'eyes_recorded': 'monocular',
  'short_name': 'MTABLER'},
 'pupil_data_type': 'AREA',
 'version_number': '5.12',
 'model': 'EyeLink 1000 Plus',
 'datetime': datetime.datetime(2025, 1, 17, 8, 38, 11),
 'calibrations': [{'timestamp': '643197.0',
   'num_points': '9',
   'type': 'P-CR',
   'tracked_eye': 'RIGHT'},
  {'timestamp': '749866.0',
   'num_points': '9',
   'type': 'P-CR',
   'tracked_eye': 'RIGHT'},
  {'timestamp': '875278.0',
   'num_points': '9',
   'type': 'P-CR',
   'tracked_eye': 'RIGHT'},
  {'timestamp': '1149430.0',
   'num_points': '9',
   'type': 'P-CR',
   'tracked_ey

In [5]:
sanity.completed_stimuli 

timestamp_started,timestamp_completed,trial_id,stimulus_id,stimulus_name,completed
f64,f64,str,i64,str,i64
1.2829e6,1.4135e6,"""PRACTICE_1""",13,"""Enc_WikiMoon""",1
1.4135e6,1.4814e6,"""PRACTICE_2""",7,"""Lit_NorthWind""",1
1.5837e6,1.8951e6,"""1""",2,"""Ins_HumanRights""",1
1.9272e6,2.3711e6,"""2""",1,"""PopSci_MultiplEYE""",1
2.4341e6,2.6633e6,"""3""",4,"""Lit_Alchemist""",1
…,…,…,…,…,…
4.2871e6,4.6624e6,"""6""",11,"""Arg_PISARapaNui""",1
4.7201e6,5.2212e6,"""7""",8,"""Lit_Solaris""",1
5.3177e6,5.5824e6,"""8""",12,"""PopSci_Caveman""",1


In [40]:
stimulus_name = "PopSci_MultiplEYE"

stimulus = Stimulus.load(sanity002.stimulus_file_path, sanity.lang, sanity.country, sanity.labnum, stimulus_name)
print(stimulus)

Stimulus(id=1, name='PopSci_MultiplEYE', type='experiment', pages=[StimulusPage(number=1, text='MultiplEYE projekt\n\nNimi „MultiplEYE“ on sõnamäng, mis ühendab sõnad „multilingualism“ (mitmekeelsus) või „multiple languages“ (mitu keelt) ja „eye“ (silm) sõnast „eye-tracking“ (silmaseire). MultiplEYE on COST aktsioon, mida rahastab Euroopa Liit. COST aktsioonid on teadusvõrgustikud, mida toetab „European Cooperation in Science and Technology“ (Euroopa teaduse ja tehnoloogia koostöövõrgustik) ehk lühidalt COST. COST toetab kasvavat teadlaste võrgustikku üle Euroopa ja kaugemal, pakkudes rahalist toetust erinevate koostöö tegevuste läbiviimiseks. ', image_path=WindowsPath('C:/Users/saphi/PycharmProjects/multipleye-preprocessing/data/stimuli_MultiplEYE_ET_EE_Tartu_1_2025/stimuli_images_ET_EE_1/popsci_multipleye_id1_page_1_ET.png')), StimulusPage(number=2, text='Nende tegevuste hulka kuuluvad töögruppide kohtumised, koolitused oskuste jagamiseks noorte teadlastega ja teadustöö külastused. M

In [11]:
import importlib
stimulus_config_spec = importlib.util.spec_from_file_location(
        "stimulus_config", sanity.experiment_config
    )

stimulus_config = importlib.util.module_from_spec(stimulus_config_spec)
stimulus_config_spec.loader.exec_module(stimulus_config)

stimulus_config.IMAGE_WIDTH_PX

1584

In [6]:
gaze = sanity.get_frame()
print(gaze)

Experiment(sampling_rate=1000.00, screen=Screen(width_px=1585, height_px=1199, width_cm=37, height_cm=28, distance_cm=60.00, origin=upper left), eyetracker=EyeTracker(sampling_rate=1000.0, left=False, right=True, model='EyeLink 1000 Plus', version='5.12', vendor='EyeLink', mount='Desktop'))
shape: (3_095_324, 10)
┌─────────┬────────┬────────────┬────────────┬───┬──────────┬────────────┬────────────┬────────────┐
│ time    ┆ pupil  ┆ screen     ┆ trial      ┆ … ┆ practice ┆ pixel      ┆ position   ┆ velocity   │
│ ---     ┆ ---    ┆ ---        ┆ ---        ┆   ┆ ---      ┆ ---        ┆ ---        ┆ ---        │
│ i64     ┆ f64    ┆ str        ┆ str        ┆   ┆ bool     ┆ list[f64]  ┆ list[f64]  ┆ list[f64]  │
╞═════════╪════════╪════════════╪════════════╪═══╪══════════╪════════════╪════════════╪════════════╡
│ 1321949 ┆ 1379.0 ┆ page_1     ┆ PRACTICE_t ┆ … ┆ true     ┆ [86.9,     ┆ [-15.34050 ┆ [0.268495, │
│         ┆        ┆            ┆ rial_1     ┆   ┆          ┆ 139.8]     ┆ 9, -

In [7]:
gaze.frame.filter(pl.col("activity") == "question").unique("screen")


time,pupil,screen,trial,stimulus,activity,practice,pixel,position,velocity
i64,f64,str,str,str,str,bool,list[f64],list[f64],list[f64]
3328517,1136.0,"""question_6111""","""trial_5""","""Lit_MagicMountain_6""","""question""",false,"[793.9, 417.0]","[0.042354, -4.051882]","[-3.296553, -6.778711]"
2673049,1096.0,"""question_4132""","""trial_3""","""Lit_Alchemist_4""","""question""",false,"[279.8, 949.4]","[-11.270187, 7.766104]","[-21.230587, -84.309272]"
1791053,1095.0,"""question_2112""","""trial_1""","""Ins_HumanRights_2""","""question""",false,"[254.3, 453.1]","[-11.81586, -3.25012]","[0.241412, 0.4637]"
2284587,1071.0,"""question_1121""","""trial_2""","""PopSci_MultiplEYE_1""","""question""",false,"[498.6, 572.4]","[-6.512206, -0.593166]","[0.829272, -0.541199]"
6277332,1150.0,"""question_9111""","""trial_10""","""Lit_BrokenApril_9""","""question""",false,"[247.4, 445.7]","[-11.963144, -3.414584]","[-0.416771, -0.578715]"
…,…,…,…,…,…,…,…,…,…
5216061,1044.0,"""question_8122""","""trial_7""","""Lit_Solaris_8""","""question""",false,"[864.2, 895.9]","[1.60904, 6.591715]","[-0.659968, 0.047564]"
3367845,1021.0,"""question_6132""","""trial_5""","""Lit_MagicMountain_6""","""question""",false,"[866.4, 550.4]","[1.658042, -1.083664]","[0.237846, -1.610259]"
6346885,1004.0,"""question_9132""","""trial_10""","""Lit_BrokenApril_9""","""question""",false,"[1000.3, 337.6]","[4.633242, -5.809303]","[-0.261754, 0.055937]"


In [12]:
gaze.frame.filter(pl.col("stimulus") == "PopSci_Caveman_12").filter(pl.col("activity") == "rating").unique(pl.col("screen"))

time,pupil,screen,trial,stimulus,activity,practice,pixel,position,velocity
i64,f64,str,str,str,str,bool,list[f64],list[f64],list[f64]
5536232,1150.0,"""familiarity_rating_screen_2""","""trial_8""","""PopSci_Caveman_12""","""rating""",false,"[174.4, 453.0]","[-13.511227, -3.252343]","[0.731723, -1.837682]"
5539720,1153.0,"""subject_difficulty_screen""","""trial_8""","""PopSci_Caveman_12""","""rating""",false,"[201.5, 333.7]","[-12.938745, -5.89537]","[-0.049032, 0.152165]"
5533404,1030.0,"""familiarity_rating_screen_1""","""trial_8""","""PopSci_Caveman_12""","""rating""",false,"[1404.9, 1000.5]","[13.412134, 8.881728]","[-0.646085, 1.277143]"


In [8]:
### inspect Metadata
gaze._metadata

{'weekday': 'Thu',
 'month': 'Jan',
 'day': 16,
 'time': '13:38:14',
 'year': 2025,
 'version_1': 'EYELINK II 1',
 'version_2': 'EYELINK II CL v5.12 May 12 2017',
 'resolution': (1585, 1199),
 'tracking_mode': 'CR',
 'sampling_rate': 1000.0,
 'file_sample_filter': '2',
 'link_sample_filter': '1',
 'tracked_eye': 'R',
 'mount_configuration': {'mount_type': 'Desktop',
  'head_stabilization': 'stabilized',
  'eyes_recorded': 'monocular',
  'short_name': 'MTABLER'},
 'pupil_data_type': 'AREA',
 'version_number': '5.12',
 'model': 'EyeLink 1000 Plus',
 'datetime': datetime.datetime(2025, 1, 16, 13, 38, 14),
 'calibrations': [{'timestamp': '747930.0',
   'num_points': '9',
   'type': 'P-CR',
   'tracked_eye': 'RIGHT'},
  {'timestamp': '766667.0',
   'num_points': '9',
   'type': 'P-CR',
   'tracked_eye': 'RIGHT'},
  {'timestamp': '952782.0',
   'num_points': '9',
   'type': 'P-CR',
   'tracked_eye': 'RIGHT'},
  {'timestamp': '1030075.0',
   'num_points': '9',
   'type': 'P-CR',
   'tracked_e

In [33]:
for part in [sanity, sanity002, sanity003]:
    gaze = part.get_frame()
    report_file_2 = open(f"{part.report_file}", "w", encoding="utf-8")
    report = partial(report_meta, report_file=report_file_2)
    check_gaze(gaze, report)    
    check_metadata(gaze._metadata, report)
    report_file_2.close()
    report_file_2 = open(f"{sanity.report_file}", "r", encoding="utf-8")
    print(report_file_2.readlines())

['✅ Number of practice trials: 2\n', '✅ Number of trials: 10\n', '❌ Number of calibrations: 14\n', '✅ AVG validation scores: 0.28, 0.14, 0.19, 0.2, 0.39, 0.16, 0.25, 0.27, 0.15, 0.15, 0.28, 0.14, 0.52, 0.15, 0.19, 0.39, 0.15, 0.34, 0.45\n', '❌ MAX validation scores: 0.82, 0.89, 0.91, 0.78, 0.94, 0.26, 0.36, 0.58, 0.35, 0.85, 0.71, 0.61, 1.06, 0.71, 0.64, 0.63, 0.41, 0.5, 0.8\n', '✅ Validation errors: GOOD, GOOD, GOOD, GOOD, GOOD, GOOD, GOOD, GOOD, GOOD, GOOD, GOOD, GOOD, GOOD, GOOD, GOOD, GOOD, GOOD, GOOD, GOOD\n', '✅ tracked_eye: R\n', '✅ Validation tracked Eyes: R, R, R, R, R, R, R, R, R, R, R, R, R, R, R, R, R, R, R\n', '✅ Data loss ratio: 0.000030%\n', '✅ Data loss ratio due to blinks: 0.000000%\n', '❌ Total recording duration: 55.489016666666664\n', '✅ Sampling rate: 1000.0\n']
['✅ Number of practice trials: 2\n', '✅ Number of trials: 10\n', '❌ Number of calibrations: 14\n', '✅ AVG validation scores: 0.28, 0.14, 0.19, 0.2, 0.39, 0.16, 0.25, 0.27, 0.15, 0.15, 0.28, 0.14, 0.52, 0.15

In [9]:
report_file_2 = open(f"{sanity.report_file}", "w", encoding="utf-8")
report = partial(report_meta, report_file=report_file_2)
check_gaze(gaze, report)    
check_metadata(gaze._metadata, report)
report_file_2.close()

In [10]:
report_file_2 = open(f"{sanity.report_file}", "r", encoding="utf-8")
report_file_2.readlines()

['✅ Number of practice trials: 2\n',
 '✅ Number of trials: 10\n',
 '❌ Date: 13:38:14;     16.16.2025\n',
 '❌ Number of calibrations: 14\n',
 '❌ Number of validations: 19\n',
 '❌ AVG validation scores: 0.28, 0.14, 0.19, 0.2, 0.39, 0.16, 0.25, 0.27, 0.15, 0.15, 0.28, 0.14, 0.52, 0.15, 0.19, 0.39, 0.15, 0.34, 0.45\n',
 '❌ MAX validation scores: 0.82, 0.89, 0.91, 0.78, 0.94, 0.26, 0.36, 0.58, 0.35, 0.85, 0.71, 0.61, 1.06, 0.71, 0.64, 0.63, 0.41, 0.5, 0.8\n',
 '✅ Validation errors: GOOD, GOOD, GOOD, GOOD, GOOD, GOOD, GOOD, GOOD, GOOD, GOOD, GOOD, GOOD, GOOD, GOOD, GOOD, GOOD, GOOD, GOOD, GOOD\n',
 '✅ tracked_eye: R\n',
 '✅ Validation tracked Eyes: R, R, R, R, R, R, R, R, R, R, R, R, R, R, R, R, R, R, R\n',
 '✅ Data loss ratio: 0.000030%\n',
 '✅ Data loss ratio due to blinks: 0.000000%\n',
 '❌ Total recording duration: 55.489016666666664\n',
 '✅ Sampling rate: 1000.0\n']

### preprocess gaze data to create event dataframe, which is used for plotting


In [18]:
f gaze.events

SyntaxError: invalid syntax (2960386090.py, line 1)

In [16]:
plot_main_sequence(gaze.events, sanity.plot_dir)

In [11]:
check_events(gaze, report) # not implemented yet   

In [19]:
print(gaze.events.frame)


shape: (27_309, 10)
┌────────────┬───────────┬───────────┬──────────┬───┬──────────┬───────────┬───────────┬───────────┐
│ trial      ┆ stimulus  ┆ screen    ┆ name     ┆ … ┆ duration ┆ location  ┆ amplitude ┆ peak_velo │
│ ---        ┆ ---       ┆ ---       ┆ ---      ┆   ┆ ---      ┆ ---       ┆ ---       ┆ city      │
│ str        ┆ str       ┆ str       ┆ str      ┆   ┆ i64      ┆ list[f64] ┆ f64       ┆ ---       │
│            ┆           ┆           ┆          ┆   ┆          ┆           ┆           ┆ f64       │
╞════════════╪═══════════╪═══════════╪══════════╪═══╪══════════╪═══════════╪═══════════╪═══════════╡
│ PRACTICE_t ┆ Enc_WikiM ┆ page_1    ┆ fixation ┆ … ┆ 584      ┆ [88.75931 ┆ null      ┆ null      │
│ rial_1     ┆ oon_13    ┆           ┆          ┆   ┆          ┆ 6, 142.72 ┆           ┆           │
│            ┆           ┆           ┆          ┆   ┆          ┆ 6667]     ┆           ┆           │
│ PRACTICE_t ┆ Enc_WikiM ┆ page_1    ┆ fixation ┆ … ┆ 210      ┆ [137.5

### Loading stimuli

In [12]:
stimuli, labconfig = load_stimuli(sanity.stimulus_file_path, sanity.lang, sanity.country, sanity.labnum)
print(stimuli)

[Stimulus(id=1, name='PopSci_MultiplEYE', type='experiment', pages=[StimulusPage(number=1, text='MultiplEYE projekt\n\nNimi „MultiplEYE“ on sõnamäng, mis ühendab sõnad „multilingualism“ (mitmekeelsus) või „multiple languages“ (mitu keelt) ja „eye“ (silm) sõnast „eye-tracking“ (silmaseire). MultiplEYE on COST aktsioon, mida rahastab Euroopa Liit. COST aktsioonid on teadusvõrgustikud, mida toetab „European Cooperation in Science and Technology“ (Euroopa teaduse ja tehnoloogia koostöövõrgustik) ehk lühidalt COST. COST toetab kasvavat teadlaste võrgustikku üle Euroopa ja kaugemal, pakkudes rahalist toetust erinevate koostöö tegevuste läbiviimiseks. ', image_path=WindowsPath('C:/Users/saphi/PycharmProjects/multipleye-preprocessing/data/stimuli_MultiplEYE_ET_EE_Tartu_1_2025/stimuli_images_ET_EE_1/popsci_multipleye_id1_page_1_ET.png')), StimulusPage(number=2, text='Nende tegevuste hulka kuuluvad töögruppide kohtumised, koolitused oskuste jagamiseks noorte teadlastega ja teadustöö külastused. 

### Plotting


In [22]:

for stimulus in stimuli:
    print(f"Plotting {stimulus.name}...")
    plot_gaze(gaze, stimulus, sanity.plot_dir)


Plotting PopSci_MultiplEYE...
Plotting Ins_HumanRights...
Plotting Ins_LearningMobility...
Plotting Lit_Alchemist...
Plotting Lit_MagicMountain...
Plotting Lit_Solaris...
Plotting Lit_BrokenApril...
Plotting Arg_PISACowsMilk...
Plotting Arg_PISARapaNui...
Plotting PopSci_Caveman...
Plotting Enc_WikiMoon...
Plotting Lit_NorthWind...


### Loading Logfile

In [13]:
sanity.logfile

timestamp,trial_number,stimulus_number,page_number,screen_onset_timestamp,keypress_timestamp,key_pressed,question,answer_correct,message
f64,str,str,str,str,str,str,str,str,str
17902.7676,"""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""*** DATE:2025-01-16"""
17904.4104,"""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""*** EXP_START_TIMESTAMP:173702…"
17906.0174,"""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""*** SESSION_ID:1"""
17907.4727,"""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""*** PARTICIPANT_ID:001"""
17909.0296,"""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""*** DATASET_TYPE:pilot_session…"
…,…,…,…,…,…,…,…,…,…
6.3128e6,"""10""","""9""","""9131""","""6298343.1822999995""","""6312761.583400003""","""up""","""True""","""True""","""FINAL ANSWER: correct answer i…"
6.3270e6,"""10""","""9""","""9132""","""6326976.974200007""","""6326969.692500003""","""down""","""True""","""False""","""preliminary answer: distractor…"
6.3272e6,"""10""","""9""","""9132""","""6326976.974200007""","""6327193.561299995""","""space""","""True""","""False""","""preliminary answer: final_conf…"


### Check logfile for screens

In [14]:
def check_all_screens_logfile(logfile: pl, stimuli: Stimulus):
    """ checking if all screens, where ET data is tracked are present in the log file"""
    for stimulus in stimuli:
        print(f"Checking {stimulus.name} in Logfile")
        trial_id = logfile.filter((pl.col("stimulus_number") == f"{stimulus.id}")).item(0, "trial_number") # get the trial number for the stimulus as ratingscreens don't have an entry in the stimulus_number column
      
        stimulus_frame = logfile.filter(
              (pl.col("trial_number") == f"{trial_id}")
            )
        #print(stimulus_frame)
        # check if all pages are present        
        for page in stimulus.pages:
            if f"{page.number}" not in stimulus_frame["page_number"].to_list():
                print(f"Missing page {stimulus.name} {page.number} in Logfile")  
                sanity.report_to_file(f" {stimulus.name}: Missing page{page.number} in Logfile", sanity.report_file)
        # check if all questions are present              
        for question in stimulus.questions:
            if f"{question.id}" not in stimulus_frame["page_number"].to_list() and f"{question.id[1:]}" not in stimulus_frame["page_number"].to_list():
                print(f"{stimulus.name}: Missing question_{question.id} in Logfile")
                sanity.report_to_file(f"{stimulus.name}: Missing question_{question.id} in Logfile", sanity.report_file)
               # print(stimulus_frame["screen"])
                
        for rating in stimulus.ratings:
            if f"{rating.name}" not in stimulus_frame["page_number"].to_list():
                print(f"{stimulus.name}: Missing rating screen {rating.name}")
                sanity.report_to_file(f"{stimulus.name}: Missing rating screen {rating.name} in Logfile", sanity.report_file)
               
check_all_screens_logfile(sanity.logfile, stimuli)

Checking PopSci_MultiplEYE in Logfile
Checking Ins_HumanRights in Logfile
Checking Ins_LearningMobility in Logfile
Checking Lit_Alchemist in Logfile
Checking Lit_MagicMountain in Logfile
Checking Lit_Solaris in Logfile
Checking Lit_BrokenApril in Logfile
Checking Arg_PISACowsMilk in Logfile
Checking Arg_PISARapaNui in Logfile
Checking PopSci_Caveman in Logfile
Checking Enc_WikiMoon in Logfile
Checking Lit_NorthWind in Logfile


## check if all pages screens ect are present in ASC file

In [15]:

def check_all_screens(gaze, stimuli):
    """ checking if all screens, where ET data is tracked are present in the gaze data frame, which is based on the ASC file
    it checks for all stimuli, if all pages and questions screens are present;it does not check for valditaion, calibration, instructions, etc.
    """
    for stimulus in stimuli:
        print(f"Checking {stimulus.name}")
        stimulus_frame = gaze.frame.filter(
              (pl.col("stimulus") == f"{stimulus.name}_{stimulus.id}")
            ).unique("screen")
        # check if all pages are present        
        for page in stimulus.pages:
            if f"page_{page.number}" not in stimulus_frame["screen"].to_list():
                print(f"Missing page {page.number}")  
                sanity.report_to_file(f"Missing page {page.number} in asc file", sanity.report_file)
        # check if all questions are present              
        for question in stimulus.questions:
            if f"question_{question.id}" not in stimulus_frame["screen"].to_list() and f"question_{question.id[1:]}" not in stimulus_frame["screen"].to_list():
                print(f"Missing question_{question.id}")
                sanity.report_to_file(f"Missing question_{question.id} in asc file",sanity.report_file)
                print(stimulus_frame["screen"])
                
        for rating in stimulus.ratings:
            if f"{rating.name}" not in stimulus_frame["screen"].to_list():
                print(f"Missing instruction {rating.name}")
                sanity.report_to_file(f"Missing rating {rating.name} in asc file",sanity.report_file)
            else: 
                print(f"Rating {rating.name} screens found")
                                
check_all_screens(gaze, stimuli)

Checking PopSci_MultiplEYE
Rating subject_difficulty_screen screens found
Rating familiarity_rating_screen_1 screens found
Rating familiarity_rating_screen_2 screens found
Checking Ins_HumanRights
Rating subject_difficulty_screen screens found
Rating familiarity_rating_screen_1 screens found
Rating familiarity_rating_screen_2 screens found
Checking Ins_LearningMobility
Rating subject_difficulty_screen screens found
Rating familiarity_rating_screen_1 screens found
Rating familiarity_rating_screen_2 screens found
Checking Lit_Alchemist
Rating subject_difficulty_screen screens found
Rating familiarity_rating_screen_1 screens found
Rating familiarity_rating_screen_2 screens found
Checking Lit_MagicMountain
Rating subject_difficulty_screen screens found
Rating familiarity_rating_screen_1 screens found
Rating familiarity_rating_screen_2 screens found
Checking Lit_Solaris
Rating subject_difficulty_screen screens found
Rating familiarity_rating_screen_1 screens found
Rating familiarity_rating_

## Checking the order of the experiment

In [16]:
def load_messages(asc_file: Path, REGEX):
    with open(asc_file, "r", encoding="utf-8") as f:
        lines = f.readlines()
    messages = []
    for line in lines:
        match = re.match(REGEX, line)
        if match:
            messages.append(match.groupdict())
    return messages

#REGEX = r'MSG\s+(?P<timestamp>\d+[.]?\d*)\s+(?P<message>\w+)'
REGEX = r'MSG\s+(?P<timestamp>\d+[.]?\d*)\s+(?P<message>.*)'

messages = load_messages(sanity.asc_file, REGEX)

In [17]:
print(messages)

[{'timestamp': '52009.0', 'message': '!CMD select_parser_configuration 0'}, {'timestamp': '52058.0', 'message': 'var initial eye_used right'}, {'timestamp': '52165.0', 'message': 'welcome_screen'}, {'timestamp': '410925.0', 'message': 'informed_consent_screen'}, {'timestamp': '434312.0', 'message': 'start_experiment'}, {'timestamp': '434312.0', 'message': 'stimulus_order_version: 2'}, {'timestamp': '434312.0', 'message': 'showing_instruction_screen_1'}, {'timestamp': '487588.0', 'message': 'showing_instruction_screen_2'}, {'timestamp': '523972.0', 'message': 'showing_instruction_screen_3'}, {'timestamp': '541537.0', 'message': 'camera_setup_screen'}, {'timestamp': '581345.0', 'message': 'DISPLAY_COORDS 0 0 1584 1198'}, {'timestamp': '747930.0', 'message': '!CAL '}, {'timestamp': '747930.0', 'message': '!CAL Calibration points:  '}, {'timestamp': '747930.0', 'message': '!CAL -37.5, -57.0        -0,      9   '}, {'timestamp': '747930.0', 'message': '!CAL -39.4, -79.0        -0,  -2090   

In [18]:
sanity.stimuli_order

[13, 7, 2, 1, 4, 3, 6, 11, 8, 12, 10, 9]

In [61]:
# check order in ASC file based on messages
def check_instructions(messages: list, stimuli: Stimulus, sanity: Sanity):
    
    messages_only = [d.get('message') for d in messages]
    one_time_screens = ['welcome_screen', 'informed_consent_screen', 'start_experiment', 'stimulus_order_version', 'showing_instruction_screen_1', 'showing_instruction_screen_2', 'showing_instruction_screen_3', 'camera_setup_screen', 'practice_text_starting_screen', 'start_recording_PRACTICE_trial_1_stimulus_Enc_WikiMoon_13_page_1','stop_recording_PRACTICE_trial_1_stimulus_Enc_WikiMoon_13_page_1','start_recording_PRACTICE_trial_1_stimulus_Enc_WikiMoon_13_page_2','stop_recording_PRACTICE_trial_1_stimulus_Enc_WikiMoon_13_page_2','start_recording_PRACTICE_trial_1_stimulus_Enc_WikiMoon_13_page_2','start_recording_PRACTICE_trial_2_stimulus_Lit_NorthWind_7_page_1', 'stop_recording_PRACTICE_trial_2_stimulus_Lit_NorthWind_7_page_1','transition_screen', 'final_validation', 'show_final_screen', 'obligatory_break', 'obligatory_break_end']
    optional_screens = ['empty_screen', 'optional_break_screen', 'fixation_trigger:skipped_by_experimenter', 'fixation_trigger:experimenter_calibration_triggered', 'optional_break', 'optional_break_duration', 'obligatory_break']
    reoccuring_screens = ['showing_subject_difficulty_screen', 'showing_familiarity_rating_screen_1','showing_familiarity_rating_screen_2']
    
    
    def _check_validation_screen(last_index, index_next_stimulus):
        #print(last_index, index_next_stimulus)
        if "validation_before_stimulus" not in messages_only[last_index:index_next_stimulus] and "final_validation" not in messages_only[last_index:index_next_stimulus]:
            print(f"{stimulus.name}: Missing validation screen")
            sanity.report_to_file(f"{stimulus.name}: Missing validation screen in asc file", sanity.report_file)
        #else:
          #  print(f"{stimulus.name}: Validation screen found")
          #  val_index = [msg.get("message") for msg in messages[last_index:index_next_stimulus]].index("validation_before_stimulus")
           # val_timestamp =messages[last_index:index_next_stimulus][val_index].get("timestamp")
           # print("val_timestamp:", val_timestamp)
            
    def _get_stimulus(id, next_id=None):
        right_stimulus = None
        next_stimulus = None
        for stimulus in stimuli:
            if stimulus.id == id:
                right_stimulus = stimulus
            if stimulus.id == next_id:
                next_stimulus = stimulus
       
        return right_stimulus, next_stimulus
            
    def _check_instruction_screens(last_index, index_next_stimulus):
        for instruction in reoccuring_screens:
            if f"{instruction}" not in messages_only[last_index:index_next_stimulus]:
                print(f"Missing instruction {instruction}")
                sanity.report_to_file(f"Missing instruction {instruction} in asc file", sanity.report_file)
           # else:
                #print(f"Rating {instruction} screens found")
    
                
    def reoccuring_msg(trial):
        index_obligatory_break = messages_only.index("obligatory_break")
        pattern = f"_trial_{trial}_stimulus_{stimulus.name}_{stimulus.id}"
        last_index = messages_only.index(f"start_recording{pattern}_page_1")
        last_timestamp = messages[last_index].get("timestamp")
        #$print(last_timestamp)
        if next_stimulus:
            index_next_stimulus = messages_only.index(f"start_recording_trial_{trial+1}_stimulus_{next_stimulus.name}_{next_stimulus.id}_page_1")
            
            if index_obligatory_break < index_next_stimulus  and index_obligatory_break > last_index:
                break_timestamp = messages[index_obligatory_break].get("timestamp")
                sanity.report_to_file(f"{trial}: {stimulus.name}: {round(((float(break_timestamp)-float(last_timestamp))/60000), 2)} minutes", sanity.report_file)
                next_timestamp = messages[index_next_stimulus].get("timestamp")
                sanity.report_to_file(f"obligatory break: {round(((float(next_timestamp)-float(break_timestamp))/60000), 2)} minutes", sanity.report_file)      
                
            else:
                next_timestamp = messages[index_next_stimulus].get("timestamp")
                sanity.report_to_file(f"{trial}:  {stimulus.name}: {round(((float(next_timestamp)-float(last_timestamp))/60000), 2)} minutes", sanity.report_file)
            
        else:
            index_next_stimulus = len(messages_only)-1
            next_timestamp = messages[index_next_stimulus].get("timestamp")
        #print(next_timestamp)
        #print(round(((float(next_timestamp)-float(last_timestamp))/60000), 2))
            sanity.report_to_file(f"{trial}:  {stimulus.name}: {((float(next_timestamp)-float(last_timestamp))/60000):.2f} minutes", sanity.report_file)
       
                
        msg_to_find = ["start_recording","screen_image_onset", "screen_image_offset", "stop_recording"]
    
            
        for page in stimulus.pages:
            for msg in msg_to_find:
                if msg.startswith("screen"):
                    current_pattern = f"page_{msg}"
                else:
                    current_pattern = f"{msg}{pattern}_page_{page.number}"
                    
                if current_pattern not in messages_only[last_index:index_next_stimulus]:
                    print(f"{stimulus.name}: Missing {current_pattern} Messages in ASC file")
                else:
                    last_index = messages_only[last_index:index_next_stimulus].index(current_pattern) + last_index
                            
        _check_instruction_screens(last_index, index_next_stimulus)  
       
        for question in stimulus.questions:
            for msg in msg_to_find:
                if msg.startswith("screen"): # get the cleaned questio_id (removed zero at the start, if present)
                    current_pattern = f"question_{msg}"
                else:
                    if question.id.startswith("0"):
                        question_id = question.id[1:]
                    else:
                        question_id = question.id
                        
                    current_pattern = f"{msg}{pattern}_question_{question_id}"
                    
                if current_pattern not in messages_only[last_index:index_next_stimulus]:
                    print(f"{stimulus.name}: Missing {current_pattern} Messages in ASC file")
                # else:
                  #  print(f"{stimulus.name}: {current_pattern} Messages is in ASC file")
                    
                    
        _check_validation_screen(last_index, index_next_stimulus)
                
    for trial, id in enumerate(sanity.stimuli_order[2:]): # for id in vars_dict["stimulus_order]: # skip th e practice trials
        try:
            stimulus, next_stimulus = _get_stimulus(id, sanity.stimuli_order[2:][trial+1]) # also get next stimulus to check for the reoccuring screens
        except IndexError:
            print("Last stimulus")
            stimulus, next_stimulus = _get_stimulus(id, None)
            
        print(f"Checking {stimulus.name}")
        reoccuring_msg(trial+1) # add one because the first trial  has number 1 not 0 
    
                
    for one_time_screen in one_time_screens:
        if f"{one_time_screen}" not in messages_only:
            found = False
            for message in messages_only:
                if message.startswith(one_time_screen):
                    #print(f"found it {message}")
                    found = True
                    
            if not found:    
                print(f"Missing screen {one_time_screen}")
                sanity.report_to_file(f"Missing one time screen {one_time_screen} in asc file", sanity.report_file)
       # else:
        #    print(f"{one_time_screen} found")
            
    for optional_screen in optional_screens:
        
        indices = list(filter(lambda i: messages_only[i] == optional_screen, range(len(messages_only))))
        if indices:
            sanity.report_to_file(f"{messages[indices[0]]['message']} found {len(indices)} times", sanity.report_file)
        
        for index in indices:
            if optional_screen == "optional_break" or optional_screen == "obligatory_break":
                msg =  messages[index+2]
                text = msg['message'].split(' ',)[0]
                duration = float(msg['message'].split(' ',)[1])/1000
                sanity.report_to_file(f"{text} lasting {duration:.2f} seconds found at {msg['timestamp']}", sanity.report_file)
                print(f"{text} lasting {duration:.2f} seconds found at {msg['timestamp']}")
                
                
            else:
                msg =  messages[index]
                sanity.report_to_file(f"{msg['message']} found at {msg['timestamp']}", sanity.report_file)
                print(f"{msg['message']} found at {msg['timestamp']}")




In [62]:
check_instructions(messages, stimuli, sanity)

Checking Lit_Alchemist
Checking Ins_LearningMobility
Checking Ins_HumanRights
Checking PopSci_MultiplEYE
Checking Lit_Solaris
Checking PopSci_Caveman
Checking Arg_PISACowsMilk
Checking Lit_BrokenApril
Checking Lit_MagicMountain
Last stimulus
Checking Arg_PISARapaNui
fixation_trigger:experimenter_calibration_triggered found at 3165964.0
optional_break_duration: lasting 17.31 seconds found at 3442544.0
optional_break_duration: lasting 5.28 seconds found at 3989277.0
optional_break_duration: lasting 5.17 seconds found at 4322811.0
optional_break_duration: lasting 12.35 seconds found at 4810744.0
optional_break_duration: lasting 3.07 seconds found at 6169745.0
optional_break_duration: lasting 3.39 seconds found at 6587167.0
optional_break_duration: lasting 4.42 seconds found at 6940333.0
optional_break_duration: lasting 3.21 seconds found at 7220860.0
obligatory_break_duration: lasting 391.90 seconds found at 5837635.0


In [41]:
gaze._metadata

{'weekday': 'Thu',
 'month': 'Jan',
 'day': 16,
 'time': '13:38:14',
 'year': 2025,
 'version_1': 'EYELINK II 1',
 'version_2': 'EYELINK II CL v5.12 May 12 2017',
 'resolution': (1585, 1199),
 'tracking_mode': 'CR',
 'sampling_rate': 1000.0,
 'file_sample_filter': '2',
 'link_sample_filter': '1',
 'tracked_eye': 'R',
 'mount_configuration': {'mount_type': 'Desktop',
  'head_stabilization': 'stabilized',
  'eyes_recorded': 'monocular',
  'short_name': 'MTABLER'},
 'pupil_data_type': 'AREA',
 'version_number': '5.12',
 'model': 'EyeLink 1000 Plus',
 'datetime': datetime.datetime(2025, 1, 16, 13, 38, 14),
 'calibrations': [{'timestamp': '747930.0',
   'num_points': '9',
   'type': 'P-CR',
   'tracked_eye': 'RIGHT'},
  {'timestamp': '766667.0',
   'num_points': '9',
   'type': 'P-CR',
   'tracked_eye': 'RIGHT'},
  {'timestamp': '952782.0',
   'num_points': '9',
   'type': 'P-CR',
   'tracked_eye': 'RIGHT'},
  {'timestamp': '1030075.0',
   'num_points': '9',
   'type': 'P-CR',
   'tracked_e

In [51]:
def check_validations(gaze, messages):
    
    for num, validation in enumerate(gaze._metadata["validations"]):
        if validation["validation_score_avg"] < "0.305":
            continue
        else:
            print(validation["validation_score_avg"], validation["timestamp"])
            bad_val_timestamp = float(validation["timestamp"])     
            found_val = False
          
        for cal in gaze._metadata["calibrations"]:
            cal_timestamp = float(cal["timestamp"])
            if cal_timestamp > bad_val_timestamp and cal_timestamp < bad_val_timestamp + 200000:
               # print(f"Calibration after validation at timestamp {cal['timestamp']}")
                #sanity.report_to_file(f"Calibration after validation at timestamp {cal['timestamp']}", sanity.report_file)
                index_bad_val = gaze._metadata["validations"].index(validation)
                next_validation = gaze._metadata['validations'][index_bad_val+1]
                time_between = round((float(next_validation["timestamp"]) - bad_val_timestamp)/1000, 3)
                print(f"next validation, {time_between} seconds later with score {next_validation['validation_score_avg']}")
                sanity.report_to_file(f"Calibration after validation at timestamp {cal['timestamp']}.   Next validation, {time_between} seconds later with score {next_validation['validation_score_avg']}", sanity.report_file)
                found_val = True
        if not found_val:
            print(f"No calibration after validation  score {validation['validation_score_avg']}")
            sanity.report_to_file(f"No calibration after validation {num+1}/{len(gaze._metadata['validations'])} at {bad_val_timestamp} with validation score {validation['validation_score_avg']}", sanity.report_file)
          
                    
            
                
check_validations(gaze, messages)

0.45 1857881.0
Calibration after bad validation: {'timestamp': '1874327.0', 'num_points': '9', 'type': 'P-CR', 'tracked_eye': 'RIGHT'}
next validation, 30.697 seconds later: {'timestamp': '1888578.0', 'num_points': '9', 'tracked_eye': 'RIGHT', 'error': 'GOOD ERROR', 'validation_score_avg': '0.19', 'validation_score_max': '0.44'}, 30.697 seconds
0.32 2755300.0
Calibration after bad validation: {'timestamp': '2770174.0', 'num_points': '9', 'type': 'P-CR', 'tracked_eye': 'RIGHT'}
next validation, 29.464 seconds later: {'timestamp': '2784764.0', 'num_points': '9', 'tracked_eye': 'RIGHT', 'error': 'GOOD ERROR', 'validation_score_avg': '0.17', 'validation_score_max': '0.46'}, 29.464 seconds


In [43]:
gaze._metadata


{'weekday': 'Thu',
 'month': 'Jan',
 'day': 16,
 'time': '13:38:14',
 'year': 2025,
 'version_1': 'EYELINK II 1',
 'version_2': 'EYELINK II CL v5.12 May 12 2017',
 'resolution': (1585, 1199),
 'tracking_mode': 'CR',
 'sampling_rate': 1000.0,
 'file_sample_filter': '2',
 'link_sample_filter': '1',
 'tracked_eye': 'R',
 'mount_configuration': {'mount_type': 'Desktop',
  'head_stabilization': 'stabilized',
  'eyes_recorded': 'monocular',
  'short_name': 'MTABLER'},
 'pupil_data_type': 'AREA',
 'version_number': '5.12',
 'model': 'EyeLink 1000 Plus',
 'datetime': datetime.datetime(2025, 1, 16, 13, 38, 14),
 'calibrations': [{'timestamp': '747930.0',
   'num_points': '9',
   'type': 'P-CR',
   'tracked_eye': 'RIGHT'},
  {'timestamp': '766667.0',
   'num_points': '9',
   'type': 'P-CR',
   'tracked_eye': 'RIGHT'},
  {'timestamp': '952782.0',
   'num_points': '9',
   'type': 'P-CR',
   'tracked_eye': 'RIGHT'},
  {'timestamp': '1030075.0',
   'num_points': '9',
   'type': 'P-CR',
   'tracked_e

In [54]:
### Run all commands at 
id = "004"
path_asc_file = f"data\\{id}_ET_EE_1_ET1\\{id}etee1.asc"
logfile_path = f"C:\\Users\saphi\PycharmProjects\multipleye-preprocessing\data\\{id}_ET_EE_1_ET1\logfiles\EXPERIMENT_LOGFILE_1_004_2025-01-24_1737714416.txt"

stimulus_file_path = "C:\\Users\saphi\PycharmProjects\multipleye-preprocessing\data\stimuli_MultiplEYE_ET_EE_Tartu_1_2025"

local_file_part = "C:\\Users\saphi\PycharmProjects\multipleye-preprocessing"

sanity = Sanity.load(path_asc_file, local_file_part, stimulus_file_path, logfile_path)
gaze = sanity.get_frame()
stimuli, labconfig = load_stimuli(sanity.stimulus_file_path, sanity.lang, sanity.country, sanity.labnum)
messages = load_messages(sanity.asc_file, REGEX)
report_file_2 = open(f"{sanity.report_file}", "w", encoding="utf-8")
report = partial(report_meta, report_file=report_file_2)
check_gaze(gaze, report)    
check_metadata(gaze._metadata, report)
report_file_2.close()
check_all_screens_logfile(sanity.logfile, stimuli)
check_validations(gaze, messages)
check_instructions(messages, stimuli, sanity)

Match was found at 5-33: 004_ET_EE_1_ET1\004etee1.asc
Group 1 found at 5-8: 004
Group 2 found at 9-11: ET
Group 3 found at 12-14: EE
Group 4 found at 15-16: 1
Group 5 found at 17-20: ET1
Group 6 found at -1--1: None
Group 7 found at 21-29: 004etee1
C:\Users\saphi\PycharmProjects\multipleye-preprocessing
96
Checking PopSci_MultiplEYE in Logfile
Checking Ins_HumanRights in Logfile
Checking Ins_LearningMobility in Logfile
Checking Lit_Alchemist in Logfile
Checking Lit_MagicMountain in Logfile
Checking Lit_Solaris in Logfile
Checking Lit_BrokenApril in Logfile
Checking Arg_PISACowsMilk in Logfile
Checking Arg_PISARapaNui in Logfile
Checking PopSci_Caveman in Logfile
Checking Enc_WikiMoon in Logfile
Checking Lit_NorthWind in Logfile
0.32 2600117.0
Calibration after bad validation: {'timestamp': '2650945.0', 'num_points': '9', 'type': 'P-CR', 'tracked_eye': 'RIGHT'}
next validation, 85.994 seconds later: {'timestamp': '2686111.0', 'num_points': '9', 'tracked_eye': 'RIGHT', 'error': 'GOOD ERR

TypeError: string indices must be integers, not 'str'

In [53]:
plot_main_sequence(gaze.events, sanity.plot_dir)
for stimulus in stimuli:
    print(f"Plotting {stimulus.name}...")
    plot_gaze(gaze, stimulus, sanity.plot_dir)

Plotting PopSci_MultiplEYE...
Plotting Ins_HumanRights...
Plotting Ins_LearningMobility...
Plotting Lit_Alchemist...
Plotting Lit_MagicMountain...
Plotting Lit_Solaris...
Plotting Lit_BrokenApril...
Plotting Arg_PISACowsMilk...
Plotting Arg_PISARapaNui...
Plotting PopSci_Caveman...
Plotting Enc_WikiMoon...
Plotting Lit_NorthWind...
